In [1]:
using JuMP
using Cbc
using DataFrames
using Gurobi
using CSV

In [2]:
# m = Model(solver = CbcSolver())
m = Model(solver=GurobiSolver(Presolve=0))

data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_simple.csv")

v = data[:,2]

simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y1[1:102], Bin #1 if item i is taken in path 1
    y2[1:102], Bin #1 if item i is taken in path 2
end

@variables m begin
    x1[1:102,1:102], Bin #1 if direct path betweeen i and j in path 1
    x2[1:102,1:102], Bin #1 if direct path betweeen i and j in path 2
end

@variables m begin
    z1[1:102]
    z2[1:102]
end

@variables m begin
    t1[1:102,1:102]
    t2[1:102,1:102]
end

@constraint(m, y1[1] == 1)
@constraint(m, y1[102] == 1)
@constraint(m, z1[1] == 0)
@constraint(m, y2[1] == 1)
@constraint(m, y2[102] == 1)
@constraint(m, z2[1] == 0)
@constraint(m, sum(y1[i] for i in 2:101) + sum(y2[i] for i in 2:101) <= 15)
#@constraint(m, sum(x[i,1] for i in 1:102) == 0)
#@constraint(m, sum(x[102,j] for j in 1:102) == 0)

for i in 2:101
    @constraint(m, y1[i] + y2[i] <= 1)
end

for i in 1:101
    @constraint(m, sum(x1[i,j] for j in 2:102) == y1[i])
    @constraint(m, sum(x2[i,j] for j in 2:102) == y2[i])
end

for j in 2:102
    @constraint(m, sum(x1[i,j] for i in 1:101) == y1[j])
    @constraint(m, sum(x2[i,j] for i in 1:101) == y2[j])
end

for j in 2:102
    @constraint(m, sum(t1[i,j] for i in 1:102) == z1[j])
    @constraint(m, sum(t2[i,j] for i in 1:102) == z2[j])
end

for j in 1:101
    @constraint(m, sum(t1[j,k] for k in 1:102) == (z1[j] + sum(c[j,k]x[j,k] for k in 1:102)))
    @constraint(m, sum(t1[j,k] for k in 1:102) == (z1[j] + sum(c[j,k]x[j,k] for k in 1:102)))
end

for j in 1:102
    for k in 2:102
        @constraint(m, t1[j,k] >= x1[j,k])
        @constraint(m, t1[j,k] <= 180x1[j,k])
        @constraint(m, t2[j,k] >= x2[j,k])
        @constraint(m, t2[j,k] <= 180x2[j,k])
        @constraint(m, t1[j,k] + t2[j,k] <= 180)
    end
end

@objective(m, Max, sum(v[i]*(y1[i] + y2[i])  for i in 1:102))


solve(m)

#for i in 2:102
#    println(i, getvalue(y[i]))
#end

for i in 2:101
    if getvalue(y1[i]) > 0.5
        println(i, " ", getvalue(y[i]))
    end
end

for i in 1:102
    for j in 1:102
        if getvalue(x1[i,j]) > 0.5
            println("i: ", i, ", j: ", j, )
        end
    end
end

for i in 2:101
    if getvalue(y2[i]) > 0.5
        println(i, " ", getvalue(y[i]))
    end
end

for i in 1:102
    for j in 1:102
        if getvalue(x2[i,j]) > 0.5
            println("i: ", i, ", j: ", j, )
        end
    end
end

#println("ANSWER")
#println("Total Distance: ", getobjectivevalue(m))
println(x)

Academic license - for non-commercial use only
Optimize a model with 21012 rows, 21012 columns and 93022 nonzeros
Variable types: 10506 continuous, 10506 integer (10506 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [9e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 2.4900000
Variable types: 10505 continuous, 10507 integer (10506 binary)

Root relaxation: objective 1.629900e+02, 10804 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     162.9900000  162.99000  0.00%     -    0s

Explored 0 nodes (10804 simplex iterations) in 0.13 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 162.99 2.49 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.629900000000e+02, best bound 1.629900000000e+02, gap